## Example code to process FootyStats API

In [35]:
import requests
import json
import pickle
import pandas as pd

def query2table(output_list):
    column_names = output_list[0].keys()
    df = pd.DataFrame(output_list, columns = column_names) 
    
    return df


### Get league table

In [109]:
api_url = "https://api.football-data-api.com/league-tables?key=example&league_id=1625"

response = requests.get(api_url)
result = response.json()
league_tables = result['data']


In [110]:
df = query2table(league_tables['league_table'])
df

,id,seasonGoals_home,seasonConceded,seasonGoals_away,seasonConceded_away,seasonConceded_home,seasonGoals,points,ppg_overall,seasonGoalDifference,...,country,cleanName,shortHand,url,seasonURL_overall,seasonURL_home,seasonURL_away,position,zone,corrections
0,93,57,50,38,11,12,95,98,2.578947,72,...,england,Manchester City,manchester-city-fc,/clubs/manchester-city-fc-93,/clubs/manchester-city-fc-93,/clubs/manchester-city-fc-93,/clubs/manchester-city-fc-93,1,"{'name': 'UEFA Champions League', 'number': 1}",0
1,151,55,44,34,12,10,89,97,2.552632,67,...,england,Liverpool,liverpool-fc,/clubs/liverpool-fc-151,/clubs/liverpool-fc-151,/clubs/liverpool-fc-151,/clubs/liverpool-fc-151,2,"{'name': 'UEFA Champions League', 'number': 1}",0
2,152,39,36,24,27,12,63,72,1.894737,24,...,england,Chelsea,chelsea-fc,/clubs/chelsea-fc-152,/clubs/chelsea-fc-152,/clubs/chelsea-fc-152,/clubs/chelsea-fc-152,3,"{'name': 'UEFA Champions League', 'number': 1}",0
3,92,34,49,33,23,16,67,71,1.868421,28,...,england,Tottenham Hotspur,tottenham-hotspur-fc,/clubs/tottenham-hotspur-fc-92,/clubs/tottenham-hotspur-fc-92,/clubs/tottenham-hotspur-fc-92,/clubs/tottenham-hotspur-fc-92,4,"{'name': 'UEFA Champions League', 'number': 1}",0
4,59,42,47,31,35,16,73,70,1.842105,22,...,england,Arsenal,arsenal-fc,/clubs/arsenal-fc-59,/clubs/arsenal-fc-59,/clubs/arsenal-fc-59,/clubs/arsenal-fc-59,5,"{'name': 'UEFA Europa League', 'number': 2}",0
5,149,33,57,32,29,25,65,66,1.736842,11,...,england,Manchester United,manchester-united-fc,/clubs/manchester-united-fc-149,/clubs/manchester-united-fc-149,/clubs/manchester-united-fc-149,/clubs/manchester-united-fc-149,6,"{'name': None, 'number': -10}",0
6,223,28,40,19,25,21,47,57,1.500000,1,...,england,Wolverhampton Wanderers,wolverhampton-wanderers-fc,/clubs/wolverhampton-wanderers-fc-223,/clubs/wolverhampton-wanderers-fc-223,/clubs/wolverhampton-wanderers-fc-223,/clubs/wolverhampton-wanderers-fc-223,7,"{'name': None, 'number': -10}",0
7,144,30,45,24,25,21,54,54,1.421053,8,...,england,Everton,everton-fc,/clubs/everton-fc-144,/clubs/everton-fc-144,/clubs/everton-fc-144,/clubs/everton-fc-144,8,"{'name': None, 'number': -10}",0
8,108,24,47,27,28,20,51,52,1.368421,3,...,england,Leicester City,leicester-city-fc,/clubs/leicester-city-fc-108,/clubs/leicester-city-fc-108,/clubs/leicester-city-fc-108,/clubs/leicester-city-fc-108,9,"{'name': None, 'number': -10}",0
9,153,32,47,20,28,27,52,52,1.368421,-3,...,england,West Ham United,west-ham-united-fc,/clubs/west-ham-united-fc-153,/clubs/west-ham-united-fc-153,/clubs/west-ham-united-fc-153,/clubs/west-ham-united-fc-153,10,"{'name': None, 'number': -10}",0


In [172]:
#get top team names
top10 = df[["id","points","cleanName"]][:10]
top10

,id,points,cleanName
0,93,98,Manchester City
1,151,97,Liverpool
2,152,72,Chelsea
3,92,71,Tottenham Hotspur
4,59,70,Arsenal
5,149,66,Manchester United
6,223,57,Wolverhampton Wanderers
7,144,54,Everton
8,108,52,Leicester City
9,153,52,West Ham United


### Get league matches

In [100]:
api_url = "https://api.football-data-api.com/league-matches?key=example&league_id=1625"
#api_url = "https://api.football-data-api.com/team?key=example&team_id=149&include=stats"

file_path = 'league_matches.pickle'
response = requests.get(api_url)
result = response.json()
league_matches = result['data']

# Write: Open the file in binary mode
with open(file_path, 'wb') as file:
    # Serialize and write the variable to the file
    pickle.dump(league_matches, file)


In [119]:
file_path = "league_matches.pickle"
# Read: Open the file in binary mode
with open(file_path, 'rb') as file:
    # Deserialize and retrieve the variable from the file
    league_matches = pickle.load(file)

matches = query2table(league_matches)

#### Select relevant columns

In [120]:
matches.drop(matches.filter(regex='odds|over|potential|ppg|xg_|url|image|btts').columns, axis=1, inplace=True)
matches.drop(matches.filter(["matches_completed_minimum", "roundID","revised_game_week"]).columns, axis=1, inplace=True)

In [121]:
columns_select = matches.columns[:10].tolist()+['winningTeam','home_name','away_name']
matches = matches.filter(columns_select)

### Calculate cumulative points over the season per team

In [180]:
import numpy as np

def game_points(row):
    if row['winningTeam'] == -1:
        val = 0
    elif row['winningTeam'] == team_id:
        val = 3
    else:
        val = 0
    return val

for n,team in top10.iterrows():
    team_id = team.id
    matches_perteam = matches.query("homeID=={0} or awayID=={0}".format(team_id)).copy()
    matches_perteam['points'] = matches_perteam.apply(game_points, axis=1)
    team_cumulative = np.cumsum(matches_perteam.points)
    tn = team.cleanName
    tcs = pd.DataFrame(team_cumulative.tolist(), columns=[tn])
    if(n==0): final_table = pd.DataFrame(tcs, columns=[tn])
    else: 
        final_table = final_table.reset_index(drop=True)
        final_table = pd.concat([final_table, tcs],axis=1)
        
final_table.index.name="game"
final_table

,Manchester City,Liverpool,Chelsea,Tottenham Hotspur,Arsenal,Manchester United,Wolverhampton Wanderers,Everton,Leicester City,West Ham United
game,,,,,,,,,,
0,3,3,3,3,0,3,0,0,0,0
1,6,6,6,6,0,3,0,3,3,0
2,6,9,9,9,3,3,0,3,6,0
3,9,12,12,9,6,6,3,3,6,0
4,12,15,15,9,9,9,6,3,6,3
5,15,18,15,12,12,9,6,3,9,3
6,18,18,15,15,15,9,9,6,12,6
7,18,18,18,18,18,12,12,9,12,6
8,21,21,18,21,21,12,12,12,12,6


### Time for fun plots

In [184]:
import bar_chart_race as bcr
import ffmpeg

df_values, df_ranks = bcr.prepare_wide_data(final_table, steps_per_period=1, orientation='h', sort='desc')

def period_summary(values, ranks):
    top2 = values.nlargest(2)
    leader = top2.index[0]
    lead = top2.iloc[0] - top2.iloc[1]
    s = f'{leader} by {lead:.0f}'
    return {'s': s, 'x': .95, 'y': .07, 'ha': 'right', 'size': 8}

bcr.bar_chart_race(df_values, period_length=1000,
                   fixed_max=True, fixed_order=False, n_bars=10,
                   figsize=(5, 3), period_fmt='Game {x:,.0f}',
                   period_summary_func=period_summary,
                   title="Premier League 2018/2019 Race to the Top")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/bar_chart_race/_make_chart.py:889: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_values.iloc[:, 0] = df_values.iloc[:, 0].fillna(method='ffill')
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/bar_chart_race/_make_chart.py:889: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_values.iloc[:, 0] = df_values.iloc[:, 0].fillna(method='ffill')
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/bar_chart_race/_make_chart.py:286: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_yticklabels(self.df_values.columns)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-package